In [1]:
import re
import os
import time
import selenium
import pandas as pd
from selenium import webdriver
from requests_html import HTML
from selenium.webdriver.chrome.options import Options

In [21]:
#options = Options()
#options.add_argument('--headless')
browser = webdriver.Chrome()

In [3]:
base_dir = os.path.dirname(os.getcwd())
data_dir = os.path.join(base_dir,'Data')
os.makedirs(data_dir,exist_ok=True)
file_path = os.path.join(data_dir,"urls.csv")
urls = pd.read_csv(file_path)
#urls = ['https://www.amazon.in/gp/product/B07S851WX5','https://www.amazon.in/gp/product/B082FTPRSK/','https://www.amazon.in/gp/product/B0895DY6F5/','https://www.amazon.in/gp/product/B00KX0I112/','https://www.amazon.in/HyperX-Cloud-Alpha-Gaming-Headset/dp/B074NBSF9N/']

In [23]:
def scrape_data(url):
    browser.get(url)
    time.sleep(2)
    html_data = browser.find_element_by_css_selector('body').get_attribute("innerHTML")
    html_str = HTML(html=html_data)

    try:
        avbl = html_str.find("#availability")[0].text
    except:
        avbl = "Currently"
        
    title_el = html_str.find('#productTitle')[0].text
    
    data_el = []
    
    if not avbl.startswith("Currently"):
        price_el = html_str.find('#priceblock_ourprice')[0].text
        try:
            saving_el = html_str.find("#regularprice_savings")[0].text.split('(')[1].split(')')[0]
        except:
            saving_el = "0%"
        data_el.append({"Title": title_el, "Price": price_el, "Available":avbl, "Saving": saving_el})
    else:
        avbl = avbl.split('.')[0]
        data_el.append({"Title": title_el, "Price": "NA", "Available":avbl, "Saving": "NA"})
            
    return data_el

In [19]:
def get_data(urls):
    dataset = []
    for url in urls:
        dataset.append(scrape_data(url)[0])
        
    return dataset

In [24]:
urls_list = []
for i in range(urls.shape[0]):
    if urls.iloc[i]["Urls"].startswith('http'):
        urls_list.append(urls.iloc[i]["Urls"])

dataset = get_data(urls_list)
df = pd.DataFrame(dataset)
df.head(n=urls.shape[0])

,Title,Price,Available,Saving
0,Prestige Sandwich Maker PGMFD 01,"₹ 1,059.00",In stock.,29%
1,Zinq Technologies Cool Slate Five Fan Cooling ...,"₹ 1,299.00",Available to ship in 1-2 days.,35%


In [16]:
target_filepath = os.path.join(data_dir,'target.csv')
df.to_csv(target_filepath,index=False)